# Моделирование, работа с файлами и интеграция между двумя ЯП

## Подготовка

### Импорт библиотек

Анализ данных:

In [1]:
import numpy as np
import pandas as pd
from scipy.interpolate import make_interp_spline

Визуализация данных:

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

Виджеты для JupyterNotebook:

In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipyfilechooser import FileChooser

Работа с процессами:

In [4]:
import subprocess

### Глобальные переменные

In [5]:
EXE_PATH = "Demographic.Exec/bin/Release/net7.0/linux-x64/Demographic.Exec"
out = widgets.Output()

## Вспомогательные функции

`draw_male_dynamic(df_male_dynamic)`

Визуализирует график изменения населения мужского пола в виде `splinechart`.

In [6]:
def draw_male_dynamic(df_male_dynamic):
    x = df_male_dynamic["Year"]
    y = df_male_dynamic[" Count"]

    X_Y_Spline = make_interp_spline(x, y)

    X_ = np.linspace(x.min(), x.max(), 500)
    Y_ = X_Y_Spline(X_)

    
    fig, ax = plt.subplots(figsize=(10, 6))
    fig.suptitle("Изменение мужского населения")
    
    plt.plot(X_, Y_)
    plt.xlabel("Год")
    plt.ylabel("Количество (тыс. человек)")

    plt.grid()
    plt.show()

`draw_female_dynamic(df_female_dynamic)`

Визуализирет график изменения населения женского пола в виде `splinechart`.

In [7]:
def draw_female_dynamic(df_female_dynamic):
    x = df_female_dynamic["Year"]
    y = df_female_dynamic[" Count"]

    X_Y_Spline = make_interp_spline(x, y)

    X_ = np.linspace(x.min(), x.max(), 500)
    Y_ = X_Y_Spline(X_)

    
    fig, ax = plt.subplots(figsize=(10, 6))
    fig.suptitle("Изменение женского населения")
    
    plt.plot(X_, Y_)
    plt.xlabel("Год")
    plt.ylabel("Количество (тыс. человек)")

    plt.grid()
    plt.show()

`draw_united_dynamic(df_male_dynamic, df_female_dynamic)`

Визуализирует график изменения общего населения в виде `splinechart`.

In [8]:
def draw_united_dynamic(df_male_dynamic, df_female_dynamic):
    df_united_dynamic = pd.DataFrame({ "Year": df_male_dynamic["Year"], " Count": df_male_dynamic[" Count"] + df_female_dynamic[" Count"] })

    x = df_united_dynamic["Year"]
    y = df_united_dynamic[" Count"]

    X_Y_Spline = make_interp_spline(x, y)

    X_ = np.linspace(x.min(), x.max(), 500)
    Y_ = X_Y_Spline(X_)

    
    fig, ax = plt.subplots(figsize=(10, 6))
    fig.suptitle("Изменение общего населения")
    
    plt.plot(X_, Y_)
    plt.xlabel("Год")
    plt.ylabel("Количество (тыс. человек)")

    plt.grid()
    plt.show()

`draw_male_age_distribution(df_male_age_distribution)`

Визуализирует возрастной состав населения мужского пола в виде `barchart`.

In [9]:
def draw_male_age_distribution(df_male_age_distribution):
    age_groups = {0: "0-18 лет",
                  1: "19-45 лет",
                  2: "46-65 лет",
                  3: "66-100 лет"}
    df_male_age_distribution["Age group"] = pd.cut(df_male_age_distribution["Age"], bins=[0, 18, 45, 65, 100], labels=age_groups.values())
    df_age_count_grouped_male = pd.DataFrame([df_male_age_distribution.groupby("Age group", observed=True)[" Count"].sum()])

    fig, ax = plt.subplots(figsize=(10, 6))
    fig.suptitle("Возрастной состав мужского населения")

    sns.barplot(data=df_age_count_grouped_male, errorbar=None)
    ax.set(xlabel="Возрастная группа", ylabel="Количество (тыс. человек)")

    plt.show()

`draw_female_age_distribution(df_female_age_distribution)`

Визуализирует возрастной состав населения женского пола в виде `barchart`.

In [10]:
def draw_female_age_distribution(df_female_age_distribution):
    age_groups = {0: "0-18 лет",
                  1: "19-45 лет",
                  2: "46-65 лет",
                  3: "66-100 лет"}
    df_female_age_distribution["Age group"] = pd.cut(df_female_age_distribution["Age"], bins=[0, 18, 45, 65, 100], labels=age_groups.values())
    df_age_count_grouped_female = pd.DataFrame([df_female_age_distribution.groupby("Age group", observed=True)[" Count"].sum()])

    fig, ax = plt.subplots(figsize=(10, 6))
    fig.suptitle("Возрастной состав женского населения")

    sns.barplot(data=df_age_count_grouped_female, errorbar=None)
    ax.set(xlabel="Возрастная группа", ylabel="Количество (тыс. человек)")

    plt.show()

`draw()`

Визуализирует все графики используя файлы с данными.

In [12]:
def draw():
    with out:
        clear_output(True)
        df_male_dynamic = pd.read_csv("MalesDynamic.csv", delimiter=",")
        df_female_dynamic = pd.read_csv("FemalesDynamic.csv", delimiter=",")
        df_male_age_distribution = pd.read_csv("MaleAgeDistribution.csv", delimiter=",")
        df_female_age_distribution = pd.read_csv("FemaleAgeDistribution.csv", delimiter=",")

        draw_male_dynamic(df_male_dynamic)
        draw_female_dynamic(df_female_dynamic)
        draw_united_dynamic(df_male_dynamic, df_female_dynamic)
        draw_male_age_distribution(df_male_age_distribution)
        draw_female_age_distribution(df_female_age_distribution)

#draw()
#out

## Виджеты

In [13]:
wInitialAge = FileChooser(
    title = "InitialAge filepath:",
    filter_pattern = "InitialAge.csv"
)

wDeathRules = FileChooser(
    title = "DeathRules filepath:",
    filter_pattern = "DeathRules.csv"
)

wRelationshipRules = FileChooser(
    title = "RelationshipRules filepath:",
    filter_pattern = "RelationshipRules.csv"
)

wYears = widgets.IntRangeSlider(
    value=[1970, 2021],
    min=1950,
    max=2070,
    description="Годы прогноза:",
    layout = widgets.Layout(width="90%"),
    style={"description_width": "initial"}
)


wPopulation = widgets.IntSlider(
    value = 130000,
    min=100000,
    max=200000000,
    description="Популяция:",
    layout = widgets.Layout(width="90%"),
    style={"description_width": "initial"},
)

wWarnings = widgets.Label()

def on_btn_click(b):
    wWarnings.value = ""
    if None in (wInitialAge.value, wDeathRules.value, wRelationshipRules.value):
        wWarnings.value = "Укажите путь к файлам InitialAge, DeathRules и RelationshipRules"
        return
    wWarnings.value = "Я не завис, пожалуйста подождите"
    complete = subprocess.run([EXE_PATH, wInitialAge.value, wDeathRules.value, wRelationshipRules.value,
                    str(wYears.value[0]), str(wYears.value[1]),
                    str(wPopulation.value)])
    if complete.returncode == 0:
        draw()
        wWarnings.value = "Готово!"
    else:
        wWarnings.value = "Что-то пошло не так..."

wBtnDraw = widgets.Button(
    description="График"
)

wBtnDraw.on_click(on_btn_click)

vBox = widgets.VBox([wInitialAge, wDeathRules, wRelationshipRules, wYears, wPopulation, wBtnDraw, wWarnings])

display(vBox)
out

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<Figure size 1000x600 with 1 Axes>', '…